In [1]:
def cdfmaxgeo(p,t,m):
    return (1-(1-p)^(m+1))^t
def pdfmaxgeo(p,t,m):
    return cdfmaxgeo(p,t,m)-cdfmaxgeo(p,t,m-1)
    

In [2]:
def nextstateprob(x,y):
    if x ==63:
        return 1-cdfmaxgeo(2/3+0.0,2^x,62)
    return pdfmaxgeo(2/3+0.0,2^x,y)

In [3]:
tm=matrix([[nextstateprob(x,y) for x in range(64)] for y in range(64)])

In [80]:
pm=tm
for _ in range(20):
 pm = pm^2
stat=pm*vector({1:1,63:0})
print(sum(stat))
stat

0.999999999952366


(0.476563095838547, 0.290008277069172, 0.139917968194352, 0.0586770548078090, 0.0224471788438172, 0.00810421240432098, 0.00282576354047732, 0.000965620671507453, 0.000326252792938701, 0.000109544499335094, 0.0000366568819313766, 0.0000122441918654699, 4.08585723181877e-6, 1.36273835423470e-6, 4.54384351862505e-7, 1.51485734113910e-7, 5.04995072826173e-8, 1.68339168055658e-8, 5.61143674448838e-9, 1.87050189995125e-9, 6.23504803604958e-10, 2.07835570232040e-10, 6.92786571866131e-11, 2.30930253742276e-11, 7.69755676003532e-12, 2.56585251964193e-12, 8.55525570183441e-13, 2.84933826330784e-13, 9.52193095509939e-14, 3.14984028202337e-14, 1.04994676112577e-14, 3.62050607336612e-15, 1.08615182206194e-15, 3.62050607359248e-16, 3.62050607361851e-16, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.0

In [81]:
stat[15]

1.51485734113910e-7

In [47]:
unbiased=tm*vector({0:1,63:0})
unbiased

(0.666666666666667, 0.222222222222222, 0.0740740740740741, 0.0246913580246914, 0.00823045267489719, 0.00274348422496573, 0.000914494741655170, 0.000304831580551723, 0.000101610526850648, 0.0000338701756168458, 0.0000112900585389486, 3.76335284635321e-6, 1.25445094878440e-6, 4.18150316261467e-7, 1.39383438679808e-7, 4.64611462636100e-8, 1.54870487545367e-8, 5.16234954783812e-9, 1.72078318261271e-9, 5.73594394204235e-10, 1.91198168408846e-10, 6.37326857955145e-11, 2.12442285985048e-11, 7.08144654026910e-12, 2.36044517265555e-12, 7.86815057551848e-13, 2.62345700718924e-13, 8.73745520379998e-14, 2.91988655476416e-14, 9.65894031423886e-15, 3.21964677141295e-15, 1.11022302462516e-15, 3.33066907387547e-16, 1.11022302462516e-16, 1.11022302462516e-16, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 0.000000000000000, 

In [48]:
pdfmaxgeo(2/3+0.0,1,15)

4.64611462636100e-8

In [49]:
unbiased[15]

4.64611462636100e-8

In [52]:
def expectedoptions(dist):
    return sum([2^i*x for (i,x) in zip(range(64),dist)])
def expectedtail(dist):
    return sum([i*x for (i,x) in zip(range(64),dist)])
print("a single transition from ubiased gives expected tail length ",expectedtail(unbiased)," and expected options ",expectedoptions(unbiased))
print("the steady state distribution has expected tail length ", expectedtail(stat), " and expected options ",expectedoptions(stat))

a single transition from ubiased gives expected tail length  0.500000000000001  and expected options  1.99999968601354
the steady state distribution has expected tail length  0.904072888428179  and expected options  3.26106196081950


In [22]:
epochsinacentury=(5*60*24*365*1000/32)
1/epochsinacentury+0.0

1.21765601217656e-8

In [63]:
print("tail length 15 occurs every",100/(stat[15]*epochsinacentury),"years in expectation under the stationary distribution")
print("tail length 15 occurs every",100/(unbiased[15]*epochsinacentury),"years in expectation under the single transition from unbiased distribution")
print("tail length 16 occurs every",100/(stat[16]*epochsinacentury),"years in expectation under the stationary distribution")
print("tail length 16 occurs every",100/(unbiased[16]*epochsinacentury),"years in expectation under the single transition from unbiased distribution")
print("tail length 17 occurs every",100/(stat[17]*epochsinacentury),"years in expectation under the stationary distribution")
print("tail length 17 occurs every",100/(unbiased[17]*epochsinacentury),"years in expectation under the single transition from unbiased distribution")

tail length 15 occurs every 8.03809031457012 years in expectation under the stationary distribution
tail length 15 occurs every 26.2080493078638 years in expectation under the single transition from unbiased distribution
tail length 16 occurs every 24.1122354988936 years in expectation under the stationary distribution
tail length 16 occurs every 78.6241479235913 years in expectation under the single transition from unbiased distribution
tail length 17 occurs every 72.3334935202939 years in expectation under the stationary distribution
tail length 17 occurs every 235.872445461677 years in expectation under the single transition from unbiased distribution


In [85]:
expectedoptions(tm*tm*vector({16:1,63:0}))

172.837461872421

In [86]:
expectedtail(tm*tm*vector({16:1,63:0}))

6.41125360736917

In [65]:
expectedoptions(tm*vector({16:1,63:0}))

1901.01954391692

In [87]:
expectedoptions(tm*tm*tm*vector({16:1,63:0}))

36.3762992608834

In [46]:
tm.transpose()[0]==tm*vector({0:1,63:0})

True